In [27]:
import re,os
import json
import ast,nltk
import datetime
from stemming.porter2 import stem
from collections import defaultdict, Counter
from string import punctuation
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import logging
import numpy as np
import time
import operator
from gensim.models import Word2Vec,KeyedVectors
from sklearn.preprocessing import Imputer
#from blagging import BlaggingClassifier
from sklearn import svm, datasets
from sklearn.utils import shuffle
from sklearn import model_selection, metrics,cross_validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import class_weight

In [4]:
training_data = []
# Articles that has category given
with open('articles.json','r') as data_file:
    line = data_file.readline()
    while line:
        line = ast.literal_eval(line)
        training_data.append(line)
        line = data_file.readline()
    data_file.close()

In [5]:
testing_data = []
# Articles that has no category given
with open('articles_wo.json','r') as data_file:
    line = data_file.readline()
    while line:
        line = ast.literal_eval(line)
        testing_data.append(line)
        line = data_file.readline()
    data_file.close()

In [6]:
# function to parse sentences to words and remove stopwords from sentence
def sentence_to_wordlist( sentence, remove_stopwords=True ):
    text = re.sub("[^a-zA-Z]"," ",sentence)
    words = text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

# function to parse text to sentences using tokenizer mentioned above
def text_to_sentences(text,tokenizer,remove_stopwords=True):
    raw_sentences = tokenizer.tokenize(text.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append( sentence_to_wordlist( raw_sentence, \
              remove_stopwords ))
    return sentences

In [81]:
categories_dict = {'automobiles' : 1, 'business' : 2, 'computing' : 3, 'criminals' : 4, 'entertainment' : 5, 'fashion' : 6,\
                  'foods' : 7, 'health' : 8, 'lifestyle' : 9, 'pbusiness' : 10, 'pfashion' : 11, 'phealth' : 12,
                  'plifestyle' : 13, 'politics' : 14, 'ptechnology' : 15, 'ptravel' : 16, 'science' : 17, 'sports' : 18,
                  'technology' : 19, 'travel' : 20}

In [88]:
#words = []
classes = []
documents = []
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# loop through each sentence in our training data
for pattern in training_data:
    # tokenize each word in the sentence
    #words.extend(nltk.word_tokenize(pattern['desc']))
    documents += [[stem(str(word)) for word in sentence] \
                        for sentence in [text_to_sentences(pattern['desc'], tokenizer)]]
    classes.append(categories_dict[pattern['category']])
    #if pattern['category'] not in no_classes:
    #    no_classes.append(pattern['category'])

In [89]:
for pattern in testing_data:
    documents += [[stem(str(word)) for word in sentence] \
                          for sentence in [text_to_sentences(pattern['desc'],tokenizer)]]

In [90]:
def class_info(classes):
    counts = Counter(classes)
    total = sum(counts.values())
    #print counts
    counts = sorted(counts.items(), key=operator.itemgetter(1),reverse = True)
    for cls in counts:
        print "%6s : % 7d  =  %0.2f%%" %(cls[0],cls[1],float(cls[1])/total*100)
        
class_info(classes)
#print classes

    14 :    3188  =  30.75%
     5 :    1684  =  16.24%
    18 :    1487  =  14.34%
     2 :     972  =  9.37%
     4 :     776  =  7.48%
    19 :     516  =  4.98%
     8 :     397  =  3.83%
    20 :     318  =  3.07%
     3 :     302  =  2.91%
     1 :     163  =  1.57%
     6 :     158  =  1.52%
    17 :     157  =  1.51%
     9 :     130  =  1.25%
     7 :      90  =  0.87%
    10 :      15  =  0.14%
    13 :      10  =  0.10%
    11 :       2  =  0.02%
    15 :       2  =  0.02%
    12 :       1  =  0.01%
    16 :       1  =  0.01%


In [96]:
print len(documents)

10852


In [99]:
freq_classes = []
new_documents = []
for i in range(len(training_data)):
    if classes[i] not in [11,15,12,16]:
        new_documents.append(documents[i])
        if classes[i] == 13:
            freq_classes.append(11)
        elif classes[i] == 14:
            freq_classes.append(12)
        elif classes[i] == 17:
            freq_classes.append(13)
        elif classes[i] == 18:
            freq_classes.append(14)
        elif classes[i] == 19:
            freq_classes.append(15)
        elif classes[i] == 20:
            freq_classes.append(16)
        else:
            freq_classes.append(classes[i])
for i in range(len(training_data),len(documents)):
    new_documents.append(documents[i])

In [100]:
print len(new_documents)
print len(freq_classes)

10846
10363


In [68]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100
# Word vector dimensionality                      
min_word_count = 2  # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 4           # Context window size                                                                                    
downsampling = 1e-5   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)

print "Training model..."
model = Word2Vec(documents, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=False)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "100features_2minwords_4context"
model.save(model_name)
#model.save_word2vec_format(model_name,binary=False)

2017-11-04 11:16:08,653 : INFO : collecting all words and their counts
2017-11-04 11:16:08,655 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-11-04 11:16:08,818 : INFO : PROGRESS: at sentence #10000, processed 204514 words, keeping 182898 word types
2017-11-04 11:16:08,830 : INFO : collected 190316 word types from a corpus of 213321 raw words and 10846 sentences
2017-11-04 11:16:08,831 : INFO : Loading a fresh vocabulary


Training model...


2017-11-04 11:16:09,112 : INFO : min_count=2 retains 14673 unique words (7% of original 190316, drops 175643)
2017-11-04 11:16:09,114 : INFO : min_count=2 leaves 37678 word corpus (17% of original 213321, drops 175643)
2017-11-04 11:16:09,168 : INFO : deleting the raw counts dictionary of 190316 items
2017-11-04 11:16:09,174 : INFO : sample=0.001 downsamples 2 most-common words
2017-11-04 11:16:09,175 : INFO : downsampling leaves estimated 36444 word corpus (96.7% of prior 37678)
2017-11-04 11:16:09,176 : INFO : estimated required memory for 14673 words and 100 dimensions: 19074900 bytes
2017-11-04 11:16:09,240 : INFO : resetting layer weights
2017-11-04 11:16:09,403 : INFO : training model with 4 workers on 14673 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4
2017-11-04 11:16:09,404 : INFO : expecting 10846 sentences, matching count from corpus used for vocabulary survey
2017-11-04 11:16:09,842 : INFO : worker thread finished; awaiting finish of 3 more t

In [69]:
def makeFeatureVec(words,model,num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float128")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #sorted_set = set(model.sort_vocab())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [70]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float128")
    # 
    # Loop through the reviews
    for review in reviews:
    # Print a status message every 1000th review
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))

    # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, \
           num_features)
    # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [101]:
DataVecs = getAvgFeatureVecs( new_documents, model, num_features )
DataVecs = Imputer().fit_transform(DataVecs)

Review 0 of 10846


/home/dell/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in divide


Review 1000 of 10846
Review 2000 of 10846
Review 3000 of 10846
Review 4000 of 10846
Review 5000 of 10846
Review 6000 of 10846
Review 7000 of 10846
Review 8000 of 10846
Review 9000 of 10846
Review 10000 of 10846


In [105]:
X_train = np.array(DataVecs[:10363])
X_test = np.array(DataVecs[10363:])
Y_train = np.array(freq_classes)

In [108]:
class_wt = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)
class_wt = dict(enumerate(class_wt.flatten(), 1))
print class_wt

{1: 3.9735429447852759, 2: 0.66634516460905346, 3: 2.14466059602649, 4: 0.83464884020618557, 5: 0.3846125296912114, 6: 4.099287974683544, 7: 7.1965277777777779, 8: 1.6314546599496222, 9: 4.9822115384615389, 10: 43.179166666666667, 11: 64.768749999999997, 12: 0.20316420953575909, 13: 4.1253980891719744, 14: 0.43556657700067247, 15: 1.2552083333333333, 16: 2.0367531446540879}


In [109]:
clf = DecisionTreeClassifier(min_samples_leaf=5,class_weight=class_wt)
predicted = model_selection.cross_val_predict(clf, X_train, Y_train, cv=5)
metrics.accuracy_score(Y_train, predicted)

0.088873878220592492